In [ ]:
!pip install --upgrade langchain langchain-openai langchain-google-genai mysql-connector-python

##Establishing the connection for MySQL
#####Note: *Make sure the credentials are updated in env variables*

---



In [ ]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()

class MySQLService:

    def __init__(self):
        # self.host = os.getenv("DB_HOST")
        # self.user = os.getenv("DB_USER")
        # self.password = os.getenv("DB_PASSWORD")
        # self.database = os.getenv("DB_NAME")
        self.host = 'localhost'
        self.user = 'root'
        self.password = 'Appaamma@21703'
        self.database = 'L1_LANDING'

    def query(self, sql: str, params=None):
        """Safely execute SQL queries and return result as list of dicts."""
        try:
            conn = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            cursor = conn.cursor(dictionary=True)
            cursor.execute(sql, params or ())
            result = cursor.fetchall()
            conn.close()
            return result

        except Exception as e:
            return {"error": str(e)}
mysql_server = MySQLService()
mysql_server.query('select * from L1_LANDING.customers')

###Agent, based on Google Gemini
This agent will invoke users query and convert it into MySQL supporting SQL commands. Then with the help of tool, we'll fetch the result of SQL

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool

mysql_server = MySQLService()
OPENAI_API_KEY="<YOUR_API_KEY>"
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


@tool  # Custom name
def get_fromDB(input):
  """Run a sql query in mysql db"""
  mysql_server.query(input)


model = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-pro',
    temperature=0.2,
    max_tokens=1000,
    timeout=30
    )
agent = create_agent(
    model,
    tools = [get_fromDB],
    system_prompt = ("You are a data assistant"
    "generate sql from the user input and pass it to get_fromDB tool"
    "Dont get hallucinated, always use sql"
    )
    )

result = agent.invoke(
    {"messages":{"role":"user", "content": "How many tables present?"}}
)
print(result)